In [1]:
from langgraph_sdk import get_client

In [2]:
client = get_client(url="http://localhost:8123")

In [3]:
topic = "Top 5 chip providers for LLM Training"

In [4]:
schema = {
    "type": "object",
    "properties": {
        "companies": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "name": {"type": "string", "description": "Company name"},
                    "technologies": {
                        "type": "string",
                        "description": "Brief summary of key technologies used by the company",
                    },
                    "market_share": {
                        "type": "string",
                        "description": "Overview of market share for this company",
                    },
                    "future_outlook": {
                        "type": "string",
                        "description": "Brief summary of future prospects and developments in the field for this company",
                    },
                    "key_powers": {
                        "type": "string",
                        "description": "Which of the 7 Powers (Scale Economies, Network Economies, Counter Positioning, Switching Costs, Branding, Cornered Resource, Process Power) best describe this company's competitive advantage",
                    },
                },
                "required": ["name", "technologies", "market_share", "future_outlook"],
            },
            "description": "List of companies",
        }
    },
    "required": ["companies"],
}

In [5]:
from langchain_core.messages import convert_to_messages

# Thread
thread = await client.threads.create()

# Model
llm = "anthropic/claude-3-5-sonnet-20240620"
llm = "openai/gpt-4o"

# Stream
async for event in client.runs.stream(
    thread["thread_id"],
    assistant_id="agent",
    input={
        "topic": topic,
        "extraction_schema": schema,
        "configurable": {"model_name": llm},
    },
    stream_mode="values",
):
    messages = event.data.get("messages", None)
    if messages:
        print(convert_to_messages(messages)[-1])
    print("=" * 25)

content='' additional_kwargs={'additional_kwargs': {'tool_calls': [{'id': 'call_kBOf74ZI5m8vDMvLFe1YaFao', 'function': {'arguments': '{"query":"Top chip providers for LLM training 2023"}', 'name': 'search'}, 'type': 'function', 'index': 0}], 'refusal': None}, 'response_metadata': {'token_usage': {'completion_tokens': 20, 'prompt_tokens': 627, 'total_tokens': 647, 'completion_tokens_details': None}, 'model_name': 'openai/gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_1bb46167f9', 'finish_reason': 'tool_calls', 'logprobs': None}, 'example': False, 'invalid_tool_calls': [], 'usage_metadata': {'input_tokens': 627, 'output_tokens': 20, 'total_tokens': 647}} response_metadata={} id='run-9ad6c10c-9de7-42d5-aef8-7073a6c6e378-0' tool_calls=[{'name': 'search', 'args': {'query': 'Top chip providers for LLM training 2023'}, 'id': 'call_kBOf74ZI5m8vDMvLFe1YaFao', 'type': 'tool_call'}]
content='[{"url": "https://developer.nvidia.com/blog/one-giant-superchip-for-llms-recommenders-and-gnns-introdu

In [6]:
current_state = await client.threads.get_state(thread["thread_id"])

In [8]:
from IPython.display import Markdown, display


def format_llm_chip_info(data):
    markdown_text = "# Top 5 Chip Providers for LLM Training\n\n"

    for company in data["companies"]:
        markdown_text += f"""
## {company['name']}

**Key Technologies:** {company['technologies']}

**Market Share:** {company['market_share']}

**Key Powers:** {company.get('key_powers', 'Not specified')}

**Future Outlook:** {company['future_outlook']}

---
"""

    return Markdown(markdown_text)


# Display the formatted markdown
display(format_llm_chip_info(current_state["values"]["info"]))

# Top 5 Chip Providers for LLM Training


## NVIDIA

**Key Technologies:** NVIDIA GH200 Grace Hopper Superchips, NVLink technology, A100 and H100 GPUs.

**Market Share:** Dominates the GPU market for LLM training, with significant revenue growth in 2023 (279% YoY increase).

**Key Powers:** Scale Economies

**Future Outlook:** Expected to continue leading in AI hardware with advancements in superchip technology and cloud integrations.

---

## Google Cloud

**Key Technologies:** TPU v5e chips, advanced distributed training infrastructure.

**Market Share:** A significant player in the cloud services market, competing with AWS and Azure for AI and machine learning applications.

**Key Powers:** Network Economies

**Future Outlook:** Continued growth in demand for cloud-based AI solutions as businesses leverage machine learning.

---

## Intel

**Key Technologies:** Intel Gaudi 2 AI accelerators, designed for high efficiency in LLM training.

**Market Share:** Increasing share in AI accelerator market with Gaudi series chips, positioned as a strong competitor to NVIDIA.

**Key Powers:** Process Power

**Future Outlook:** Positive outlook with ongoing development of Gaudi 3 and further enhancements in AI training capabilities.

---

## AMD

**Key Technologies:** AMD MI250 and MI300X GPUs, focusing on AI and machine learning applications.

**Market Share:** Growing presence in the GPU market, particularly with their MI series chips, competing against NVIDIA.

**Key Powers:** Counter Positioning

**Future Outlook:** Anticipated growth in the AI sector as demand for efficient GPUs rises.

---

## AWS

**Key Technologies:** Integration of NVIDIA superchips into their EC2 offerings.

**Market Share:** Significant market share in cloud services, leveraging partnerships with NVIDIA for advanced AI services.

**Key Powers:** Scale Economies

**Future Outlook:** Expected to expand AI service offerings with new technologies and partnerships.

---
